In [1]:
# In a Jupyter Notebook cell
%load_ext autoreload
%autoreload 2

import numpy as np
print(np.__version__)

from mirage import WordCountingChunkingAlgorithm, FolderRawStorage, SQLiteChunkStorage, RAMChunkStorage
from mirage import HuggingFaceEmbedder, TextNormalizer
from mirage import NatashaSentenceChunking, SemanticChunking
import os
import pickle

1.26.4


c:\Users\User\Desktop\Диплом\MIRAGE\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [59]:
import logging
logging.basicConfig(level=logging.ERROR)


documents = FolderRawStorage('data_test')
chunks = RAMChunkStorage()
# algorithm = WordCountingChunkingAlgorithm(documents, chunks, words_amount=100)

algorithm = SemanticChunking(raw_storage=documents, chunk_storage=chunks, threshold=0.7, max_chunk_size=1000)
algorithm.execute(visualize=True)

1

In [60]:
index = chunks.get_indexes()[24]
print(len(chunks[index]))
chunks[index]

364


'В 55-м году он повез сына в университет; прожил с ним три зимы в Петербурге, почти никуда не выходя и стараясь заводить знакомства с молодыми товарищами Аркадия. На последнюю зиму он приехать не мог,\xa0– и вот мы видим его в мае месяце 1859 года, уже совсем седого, пухленького и немного сгорбленного: он ждет сына, получившего, как некогда он сам, звание кандидата.'

In [61]:
emb = HuggingFaceEmbedder(normalizer = True, model_name='Alibaba-NLP/gte-large-en-v1.5')
emb.fit(chunks)
emb.get_dimensionality()

1024

In [4]:
from mirage.index.vector_index.ram_vector_index import L2RAMVectorIndex
from mirage import FaissIndexFlatL2, FaissIndexIVFPQR
vector_index = FaissIndexIVFPQR(dimensionality=emb.get_dimensionality())
emb.convert_chunks_to_vector_index(chunks, vector_index, visualize=True)

Converting ChunkStorage to VectorIndex


100%|██████████| 974/974 [19:42<00:00,  1.21s/it]


In [15]:
query = 'Что отрицает Базаров?'
query_vector = emb.embed(text=query)
# print(query_vector)
answer = vector_index.query(
    query_vector, top_k=20
)
print([element.distance for element in answer])
[chunks[element.chunk_storage_key] for element in answer]

[259.71051025390625, 269.3592529296875, 298.29864501953125, 302.4664001464844, 303.02880859375, 306.7166748046875, 307.56890869140625, 308.5302734375, 308.9055480957031, 309.5325927734375, 310.83770751953125, 316.61688232421875, 319.4338073730469, 324.6951599121094, 327.32177734375, 327.8720397949219, 327.8720397949219, 328.96356201171875, 330.14581298828125, 330.72845458984375]


['– В теперешнее время полезнее всего отрицание – мы отрицаем. –\xa0Все? –\xa0Все. –\xa0Как? не только искусство, поэзию… но и… страшно вымолвить… –\xa0Все,\xa0– с невыразимым спокойствием повторил Базаров.',
 'Хотите, я вас представлю? –\xa0Пожалуй… после этой кадрили. Базаров также обратил внимание на Одинцову. –\xa0Это что за фигура?\xa0– проговорил он. – На остальных баб не похожа.',
 'Прощаясь с Базаровым, Одинцова протянула ему руку и сказала:\r\n\r\n–\xa0Мы еще увидимся, не правда ли? –\xa0Как прикажете,\xa0– ответил Базаров. –\xa0В таком случае мы увидимся. Аркадий первый вышел на крыльцо; он взобрался в ситниковскую коляску. Его почтительно подсаживал дворецкий, а он бы с удовольствием его побил или расплакался.',
 '–\xa0Нет, зачем говорить о любви,\xa0– промолвил Базаров,\xa0– а вот вы упомянули об Одинцовой… Так, кажется, вы ее назвали? Кто эта барыня? –\xa0Прелесть! прелесть!\xa0– запищал Ситников. – Я вас представлю.',
 '– Прощайте,\xa0– заговорил он опять после небольшого

In [16]:
from mirage.inference.prompters.APILLM import LLM
llm = LLM()
llm.do_request(query=query,
               chunk_storage=chunks,
               indexes=[element.chunk_storage_key for element in answer],
               prompt='')


На основе данного текста можно сделать вывод, что Базаров отрицает:

1. Искусство и поэзию;
2. Любовь.

Также стоит отметить, что Базаров явно противостоял идеям романтики и эмоциональному подходу к жизни, предпочитая более холодный и расчетливый взгляд на вещи. 

С одной стороны, он был скептически настроен по отношению к "страшному" предмету — любви, который мог вызвать слишком большие эмоции и затронуть самые чувственные области человеческой психики. Но с другой стороны, именно этот момент стал причиной его встречи со Ситниковым и началом нового знакомства с Одинцовой.

Поэтому основными точками отрицания являются искусственный интерес к этим областям и стремление к осмысленному, практичному анализу вместо эмоциональных реакций. Однако важно понимать контекст этого отрицания: Базаров не полностью исключает возможность существования таких вещей, скорее он отказывается от эмоционального восприятия и цензуры, которые часто связаны с этими категориями.
15.157871961593628
